In [64]:
import math
import pandas as pd
import numpy as np
import random

In [65]:
def distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def get_circle_radius(circle_size):
    return math.sqrt(circle_size/math.pi)

def get_square_half_side(size):
    return math.sqrt(size) / 2

def check_circle_square_collision(circle_x, circle_y, circle_size,
                                square_x, square_y, square_size): 
    
    # Вычисляем радиус круга и половину стороны квадрата
    radius = get_circle_radius(circle_size)
    half_side = get_square_half_side(square_size)
    
    # Находим ближайшую точку квадрата к центру круга
    closest_x = max(square_x - half_side, min(circle_x, square_x + half_side))
    closest_y = max(square_y - half_side, min(circle_y, square_y + half_side))
    
    # Вычисляем расстояние от центра круга до ближайшей точки
    dist = distance(circle_x, circle_y, closest_x, closest_y)
    
    # Если расстояние меньше радиуса, есть коллизия
    return dist < radius

In [66]:
def collision(obj1, obj2):
    if obj1['figure'] == 'circle' and obj2['figure'] == 'circle':
        return (distance(obj1['x'], obj2['x'], obj1['y'], obj2['y']) < get_circle_radius(obj1['size']) + get_circle_radius(obj2['size']))
    
    elif obj1['figure'] == 'rectangle' and obj2['figure'] == 'rectangle':
        return (abs(obj1['x'] - obj2['x']) < (get_square_half_side(obj1['size'])) + get_square_half_side(obj2['size'])) and abs(obj1['y'] - obj2['y']) < (get_square_half_side(obj1['size'])) + get_square_half_side(obj2['size'])
        
    elif (obj1['figure'] in ('rectangle', 'circle') and 
        obj2['figure'] in ('rectangle', 'circle') and 
        obj1['figure'] != obj2['figure']):
        # Определяем, какой объект круг, а какой прямоугольник
        circle_obj = obj1 if obj1['figure'] == 'circle' else obj2
        rect_obj = obj2 if obj1['figure'] == 'circle' else obj1
        
        # Создаем кортежи с нужными данными
        circle = (circle_obj['x'], circle_obj['y'], get_circle_radius(circle_obj['size']))
        square = (rect_obj['x'], rect_obj['y'], rect_obj['size'])
            
        result = check_circle_square_collision(circle[0], circle[1], circle[2],
                                            square[0], square[1], square[2])
        return result

In [67]:
def generate_random_object():
    return {
        'x': random.randint(-5, 5),
        'y': random.randint(-6, 6),
        'size': random.randint(10, 100),
        'figure': random.choice(['circle', 'rectangle']),

        'is_active': random.choice([0, 1]),  
        'color': random.choice(['red', 'blue', 'green', 'yellow']), 
        'priority': random.choice([1, 2, 3, 4, 5]),  
        
        'speed': random.uniform(0, 10),  
        'opacity': random.uniform(0, 1),  
        'rotation': random.randint(0, 359)  
    }

In [68]:
def generate_data(num_rows, num_features=4):
    # Определяем полный набор колонок
    all_columns = [
        'x1', 'y1', 'size1', 'figure1',  
        'is_active1', 'color1', 'priority1', 'speed1', 'opacity1', 'rotation1',  
        'x2', 'y2', 'size2', 'figure2',  
        'is_active2', 'color2', 'priority2', 'speed2', 'opacity2', 'rotation2',  
        'collision'  
    ]
    
    # Выбираем нужное количество признаков (исключая collision)
    if num_features == 4:
        selected_columns = ['x1', 'y1', 'size1', 'figure1', 
                          'x2', 'y2', 'size2', 'figure2', 'collision']
    elif num_features == 7:
        selected_columns = ['x1', 'y1', 'size1', 'figure1', 'is_active1', 'color1', 'priority1',
                          'x2', 'y2', 'size2', 'figure2', 'is_active2', 'color2', 'priority2', 'collision']
    else:  # 10 признаков
        selected_columns = all_columns

    df = pd.DataFrame(columns=selected_columns)
    
    for i in range(num_rows):
        obj1 = generate_random_object()
        obj2 = generate_random_object()
        
        # Создаем словарь с данными для строки
        row_data = {
            'x1': obj1['x'], 'y1': obj1['y'], 'size1': obj1['size'], 'figure1': obj1['figure'],
            'x2': obj2['x'], 'y2': obj2['y'], 'size2': obj2['size'], 'figure2': obj2['figure'],
            'is_active1': obj1['is_active'], 'color1': obj1['color'], 
            'priority1': obj1['priority'], 'speed1': obj1['speed'],
            'opacity1': obj1['opacity'], 'rotation1': obj1['rotation'],
            'is_active2': obj2['is_active'], 'color2': obj2['color'],
            'priority2': obj2['priority'], 'speed2': obj2['speed'],
            'opacity2': obj2['opacity'], 'rotation2': obj2['rotation'],
            'collision': collision(obj1, obj2)
        }
        
        # Фильтруем только нужные колонки
        filtered_row = {k: row_data[k] for k in selected_columns}
        temp_df = pd.DataFrame([filtered_row])
        df = pd.concat([df, temp_df], ignore_index=True)
    
    return df

In [69]:
datasets = {}

# Параметры размеров
size_ranges = [50,200,500,1000]


# Количество признаков
feature_counts = [4, 7, 10]

# Генерируем датасеты
dataset_counter = 1
for size_range in size_ranges:
    for feature_count in feature_counts:
        datasets[f'dataset_{dataset_counter}'] = generate_data(size_range, feature_count)
        print(f"Dataset {dataset_counter}: {size_range} строк, {feature_count} признаков")
        dataset_counter += 1

# Пример сохранения датасетов в файлы
for name, df in datasets.items():
    df.to_csv(f"{name}.csv", index=False)

Dataset 1: 50 строк, 4 признаков
Dataset 2: 50 строк, 7 признаков
Dataset 3: 50 строк, 10 признаков
Dataset 4: 200 строк, 4 признаков


C:\Users\user\AppData\Local\Temp\ipykernel_9740\2956248099.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, temp_df], ignore_index=True)


Dataset 5: 200 строк, 7 признаков
Dataset 6: 200 строк, 10 признаков


C:\Users\user\AppData\Local\Temp\ipykernel_9740\2956248099.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, temp_df], ignore_index=True)


Dataset 7: 500 строк, 4 признаков
Dataset 8: 500 строк, 7 признаков


C:\Users\user\AppData\Local\Temp\ipykernel_9740\2956248099.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, temp_df], ignore_index=True)


Dataset 9: 500 строк, 10 признаков
Dataset 10: 1000 строк, 4 признаков
Dataset 11: 1000 строк, 7 признаков


C:\Users\user\AppData\Local\Temp\ipykernel_9740\2956248099.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, temp_df], ignore_index=True)


Dataset 12: 1000 строк, 10 признаков


In [70]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import pickle
import time

# Предполагаем, что у нас есть один из ваших датасетов, например, dataset_1.csv
df = pd.read_csv('dataset_1.csv')

# Подготовка данных
# Кодируем категориальные признаки
le_figure1 = LabelEncoder()
le_figure2 = LabelEncoder()
df['figure1'] = le_figure1.fit_transform(df['figure1'])
df['figure2'] = le_figure2.fit_transform(df['figure2'])

# Если есть дополнительные категориальные признаки (color1, color2)
if 'color1' in df.columns:
    le_color1 = LabelEncoder()
    le_color2 = LabelEncoder()
    df['color1'] = le_color1.fit_transform(df['color1'])
    df['color2'] = le_color2.fit_transform(df['color2'])

# Разделяем признаки и целевую переменную
X = df.drop('collision', axis=1)
y = df['collision']

# Разделяем на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Выбор 4 классических методов машинного обучения
models = {
    # Логистическая регрессия: 
    # - Простой и интерпретируемый метод
    # - Хорошо работает с бинарной классификацией
    # - Эффективен для линейно разделимых данных
    'Logistic Regression': LogisticRegression(max_iter=1000),
    
    # Дерево решений:
    # - Интуитивно понятен и легко интерпретируется
    # - Может моделировать нелинейные зависимости
    # - Хорошо подходит для смешанных типов данных
    'Decision Tree': DecisionTreeClassifier(max_depth=5),
    
    # k-ближайших соседей:
    # - Простой и не требует предположений о распределении данных
    # - Эффективен для задач с локальной структурой данных
    # - Подходит для небольших и средних датасетов
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    
    # Наивный Байес:
    # - Очень быстрый и требует мало данных для обучения
    # - Хорошо работает с независимыми признаками
    # - Прост в реализации и эффективен для базовой классификации
    'Naive Bayes': GaussianNB()
}

# Обучение и оценка моделей
results = {}
for name, model in models.items():
    start_time = time.time()
    model.fit(X_train, y_train)
    training_time = time.time() - start_time
    accuracy = model.score(X_test, y_test)
    results[name] = {'accuracy': accuracy, 'time': training_time}
    
    # Сохранение модели
    with open(f'{name.lower().replace(" ", "_")}_model.pkl', 'wb') as f:
        pickle.dump(model, f)
    
    print(f"{name}: потребление = {accuracy:.4f}, Время обучения = {training_time:.4f} сек")

# Выбор 3 наиболее быстрых алгоритмов с низким потреблением данных
# Основываясь на характеристиках:
# 1. Naive Bayes - минимальное время обучения, низкие требования к данным
# 2. Logistic Regression - быстрая сходимость, работает с небольшими выборками
# 3. Decision Tree - быстрое обучение на небольших данных, не требует сложной предобработки

fastest_models = [
    'Naive Bayes',
    'Logistic Regression',
    'Decision Tree'
]

print("\nТри наиболее быстрых алгоритма с низким потреблением данных:")
for model_name in fastest_models:
    print(f"- {model_name}: потребление = {results[model_name]['accuracy']:.4f}, "
          f"время = {results[model_name]['time']:.4f} сек")

# Пример загрузки сохраненной модели
# with open('naive_bayes_model.pkl', 'rb') as f:
#     loaded_model = pickle.load(f)

Logistic Regression: потребление = 0.6000, Время обучения = 0.0101 сек
Decision Tree: потребление = 0.4000, Время обучения = 0.0012 сек
K-Nearest Neighbors: потребление = 0.4000, Время обучения = 0.0008 сек
Naive Bayes: потребление = 0.8000, Время обучения = 0.0011 сек

Три наиболее быстрых алгоритма с низким потреблением данных:
- Naive Bayes: потребление = 0.8000, время = 0.0011 сек
- Logistic Regression: потребление = 0.6000, время = 0.0101 сек
- Decision Tree: потребление = 0.4000, время = 0.0012 сек
